In [1]:
# Import packages
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
# Load the rides for a given year
which_year = '2018'
file_format = '.csv'
rides_filename = 'AggregatedTrips' + which_year + file_format
weather_filename = 'Weather' + which_year + file_format
rides_df = pd.read_csv(rides_filename)
weather_df = pd.read_csv(weather_filename)

In [3]:
# Get a little more from the data out of the precipitation
def is_rain(x):
    if x == 'rain': 
        return 1
    else: 
        return 0

def is_snow(x):
    if x == 'snow': 
        return 1
    else: 
        return 0    
        
def is_sleet(x):
    if x == 'sleet': 
        return 1
    else: 
        return 0 

def precYN(x):
    if x in ['sleet', "snow", "rain"]: 
        return 1
    else: 
        return 0 
    
def day_week(x):
    if x == 0: 
        return "Monday"
    elif x == 1: 
        return "Tuesday"
    elif x == 2: 
        return "Wednesday"
    elif x == 3: 
        return "Thursday"
    elif x == 4: 
        return "Friday"
    elif x == 5: 
        return "Saturday"
    elif x == 6: 
        return "Sunday"


In [4]:
# shape columns for join for weather data
weather_df['hour'] = pd.DatetimeIndex(weather_df['hour']).hour
weather_df['date'] = pd.to_datetime(weather_df['date_time'])
weather_df['snow'] = weather_df["precipType"].apply(is_snow)
weather_df['rain'] = weather_df["precipType"].apply(is_rain)
weather_df['sleet'] = weather_df["precipType"].apply(is_sleet)
weather_df['precYN'] = weather_df["precipType"].apply(precYN)

# set dummy variables for weekdays
weather_df['weekday_dummy'] = weather_df["weekday"].apply(day_week)
temp_dummy_week = pd.get_dummies(weather_df['weekday_dummy'])
weather_df = pd.concat([weather_df, temp_dummy_week], axis=1)

In [5]:
# TO DO: need to check why one hour is missing from 23s
# categorize hours
bins = [0, 6, 10, 16, 20]
names = ['night', 'rushM', 'day', 'rushE', 'evening']
temp_d = dict(enumerate(names, 1))
weather_df['time_day'] = np.vectorize(temp_d.get)(np.digitize(weather_df['hour'], bins))

temp_dummy_day = pd.get_dummies(weather_df['time_day'])
weather_df = pd.concat([weather_df, temp_dummy_day], axis=1)

In [6]:
# shape columns for join for rides data
rides_df['hour'] = rides_df['start_time_hour']
rides_df['date'] = pd.to_datetime(rides_df['start_time_date']) 

In [7]:
# merge
new_df = pd.merge(rides_df, weather_df, on = ['date', 'hour'])

In [8]:
result_filename = 'FullData' + which_year + file_format
new_df.to_csv(result_filename)